In [1]:
import json
import pandas as pd
try:
    import requests
except:
    !pip install requests
    import requests

try:
  from tqdm import tqdm
except:
  !pip install tqdm
  from tqdm import tqdm

In [2]:
# Playlists
df_spotify = pd.read_csv('../data/2000_spotify_sample.csv.gz')
df_spotify

/home/chr/tmp/dl/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,num_edits,duration_ms,num_artists,description,pos,artist_name,track_uri,artist_uri,track_name,album_uri,album_name
0,music,False,521,1506643200,127,108,1,45,219480,85,NaN,0.0,Charli XCX,spotify:track:0y8Wf6ltWpGCpgqVa21QNX,spotify:artist:25uiPmTg16RbhZWAqwLBy5,After The Afterparty (feat. Lil Yachty),spotify:album:4Txd0wc4WAgnoxXx0SLAlw,After The Afterparty (feat. Lil Yachty)
1,music,False,521,1506643200,127,108,1,45,247160,85,NaN,1.0,The Chainsmokers,spotify:track:6RUKPb4LETWmmr3iAEQktW,spotify:artist:69GGBxA162lTqCwzJG5jLp,Something Just Like This,spotify:album:4JPguzRps3kuWDD5GS6oXr,Memories...Do Not Open
2,music,False,521,1506643200,127,108,1,45,223846,85,NaN,2.0,MAX,spotify:track:1sYSP7gKa5kdKIfhANfori,spotify:artist:1bqxdqvUtPWZri43cKHac8,Lights Down Low,spotify:album:2qmysBdftCYXRjYHTVK3qo,Hell's Kitchen Angel
3,music,False,521,1506643200,127,108,1,45,220293,85,NaN,3.0,Post Malone,spotify:track:3a1lNhkSLSkpJE4MSHpDu9,spotify:artist:246dkjvS1zLTtiykXe5h60,Congratulations,spotify:album:5s0rmjP8XOPhP6HhqOhuyC,Stoney
4,music,False,521,1506643200,127,108,1,45,201160,85,NaN,4.0,AJR,spotify:track:3E2Zh20GDCR9B1EYjfXWyv,spotify:artist:6s22t5Y3prQHyaHWUN1R1C,Weak,spotify:album:7LACXphpLTluKLFqHIZ1Qq,The Click
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131675,Fresh,False,999737,1505347200,9,9,1,6,222426,6,NaN,4.0,Iggy Azalea,spotify:track:4AEVXHznFXkJEtFamkYwlr,spotify:artist:5yG7ZAZafVaAlMTeBybKAL,Mo Bounce,spotify:album:3F0NEDBkjck7r8MMg7kniD,Mo Bounce
131676,Fresh,False,999737,1505347200,9,9,1,6,223196,6,NaN,5.0,Iggy Azalea,spotify:track:3yx6eryOZgO54bt3B671cn,spotify:artist:5yG7ZAZafVaAlMTeBybKAL,Work,spotify:album:3r51pg8BwnjazLJlW6dJIh,The New Classic
131677,Fresh,False,999737,1505347200,9,9,1,6,220748,6,NaN,6.0,Iggy Azalea,spotify:track:1QJLm4mdmqRkXXbHc4pYWY,spotify:artist:5yG7ZAZafVaAlMTeBybKAL,Change Your Life,spotify:album:6lBTYoX7ZzU2Xbf8RHAv0u,The New Classic
131678,Fresh,False,999737,1505347200,9,9,1,6,218746,6,NaN,7.0,Hailee Steinfeld,spotify:track:3Bjr9MzHM7KHk6zq7KvJRN,spotify:artist:5p7f24Rk5HkUZsaS3BLG5F,Love Myself,spotify:album:5m01tty1948HnkXUhG0urw,HAIZ


In [4]:
# Se crean las querys para obtener respuesta de la API.
# https://developer.spotify.com/documentation/web-api/reference/#

set_uris = df_spotify.artist_uri.unique()
querys = []
spotify_ids = ""
max_ids = 50 # Cantidad máxima de ids por query (depende del endpoint)

for idx, track in enumerate(set_uris, start=1):
    spotify_ids += track.split(":")[-1]
    if idx%max_ids == 0:
        querys.append(spotify_ids)
        spotify_ids = ""
        continue
    spotify_ids += ","

if spotify_ids != '':
    querys.append(spotify_ids[:-1])
len(querys)

294

# Credenciales

In [5]:
# https://developer.spotify.com/documentation/general/guides/authorization/client-credentials/
# Pide una autorización 'Basic' con: <client_id + : + client_secret> en base64. 
import base64


client_id = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
client_secret = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
usr_pass = client_id + ':' + client_secret
auth_b64 = base64.b64encode(usr_pass.encode()).decode()


url = "https://accounts.spotify.com/api/token"

payload='grant_type=client_credentials'
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  'Authorization': 'Basic ' + auth_b64
}

response = requests.request("POST", url, headers=headers, data=payload)

credentials = response.json()
credentials

{'access_token': 'BQDb5yZ5-Ckcr4zXkQR1lkQ2KLCdKgWx42jrevqjnkAMDXly9pbUiSuyKJwDT-KGnK-AvfC8d2IaqrgrDJ8',
 'token_type': 'Bearer',
 'expires_in': 3600}

# Utilizando la API

In [6]:
headers = {
  'Content-Type': 'application/json',
  'Authorization': credentials['token_type'] + ' ' + credentials['access_token']
}

file_name = "artist_sample_test_2"
URL = "https://api.spotify.com/v1/artists?ids="

# se guardan las respuestas en un .json
with open('../data/{}.json'.format(file_name), 'w') as f:
    f.write('[ \n')
    for idx, query in enumerate(tqdm(querys), start=1):
        try:
            response = requests.request("GET", URL + query, headers=headers)
            json.dump(response.json(), f, indent=2)
            if idx < len(querys):
                f.write(', \n')
        except Exception as err:
            console.log('Error: ', err)
    f.write('] \n')

100%|██████████| 294/294 [01:04<00:00,  4.58it/s]


In [7]:
# Ojo con el path.
with open('../data/{}.json'.format(file_name), 'r') as f:
    json_file = json.load(f)

In [8]:
df_spotify = pd.DataFrame(json_file)
df_spotify

,artists
0,[{'external_urls': {'spotify': 'https://open.s...
1,[{'external_urls': {'spotify': 'https://open.s...
2,[{'external_urls': {'spotify': 'https://open.s...
3,[{'external_urls': {'spotify': 'https://open.s...
4,[{'external_urls': {'spotify': 'https://open.s...
...,...
289,[{'external_urls': {'spotify': 'https://open.s...
290,[{'external_urls': {'spotify': 'https://open.s...
291,[{'external_urls': {'spotify': 'https://open.s...
292,[{'external_urls': {'spotify': 'https://open.s...


In [9]:
df_explode = df_spotify.explode('artists').reset_index()
df_explode

,index,artists
0,0,{'external_urls': {'spotify': 'https://open.sp...
1,0,{'external_urls': {'spotify': 'https://open.sp...
2,0,{'external_urls': {'spotify': 'https://open.sp...
3,0,{'external_urls': {'spotify': 'https://open.sp...
4,0,{'external_urls': {'spotify': 'https://open.sp...
...,...,...
14695,293,{'external_urls': {'spotify': 'https://open.sp...
14696,293,{'external_urls': {'spotify': 'https://open.sp...
14697,293,{'external_urls': {'spotify': 'https://open.sp...
14698,293,{'external_urls': {'spotify': 'https://open.sp...


In [10]:
df_json = df_explode.to_json(orient='records')
df_final = pd.json_normalize(json.loads(df_json), meta=['artists'])
df_final

,index,artists.external_urls.spotify,artists.followers.href,artists.followers.total,artists.genres,artists.href,artists.id,artists.images,artists.name,artists.popularity,artists.type,artists.uri
0,0,https://open.spotify.com/artist/25uiPmTg16RbhZ...,None,2594395,"[art pop, candy pop, dance pop, electropop, me...",https://api.spotify.com/v1/artists/25uiPmTg16R...,25uiPmTg16RbhZWAqwLBy5,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Charli XCX,83,artist,spotify:artist:25uiPmTg16RbhZWAqwLBy5
1,0,https://open.spotify.com/artist/69GGBxA162lTqC...,None,18920191,"[dance pop, edm, electropop, pop, pop dance, t...",https://api.spotify.com/v1/artists/69GGBxA162l...,69GGBxA162lTqCwzJG5jLp,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",The Chainsmokers,85,artist,spotify:artist:69GGBxA162lTqCwzJG5jLp
2,0,https://open.spotify.com/artist/1bqxdqvUtPWZri...,None,736574,"[dance pop, electropop, pop, post-teen pop, vi...",https://api.spotify.com/v1/artists/1bqxdqvUtPW...,1bqxdqvUtPWZri43cKHac8,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",MAX,74,artist,spotify:artist:1bqxdqvUtPWZri43cKHac8
3,0,https://open.spotify.com/artist/246dkjvS1zLTti...,None,36359357,"[dfw rap, melodic rap, rap]",https://api.spotify.com/v1/artists/246dkjvS1zL...,246dkjvS1zLTtiykXe5h60,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Post Malone,91,artist,spotify:artist:246dkjvS1zLTtiykXe5h60
4,0,https://open.spotify.com/artist/6s22t5Y3prQHya...,None,2155317,[modern rock],https://api.spotify.com/v1/artists/6s22t5Y3prQ...,6s22t5Y3prQHyaHWUN1R1C,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",AJR,78,artist,spotify:artist:6s22t5Y3prQHyaHWUN1R1C
...,...,...,...,...,...,...,...,...,...,...,...,...
14695,293,https://open.spotify.com/artist/3F774c1w8i3xFi...,None,896,[],https://api.spotify.com/v1/artists/3F774c1w8i3...,3F774c1w8i3xFiknTAJyqy,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Petey Quezada,23,artist,spotify:artist:3F774c1w8i3xFiknTAJyqy
14696,293,https://open.spotify.com/artist/3fD1A7V9jV0eIm...,None,31816,[corridos alternativos],https://api.spotify.com/v1/artists/3fD1A7V9jV0...,3fD1A7V9jV0eImdBL0Bcux,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Los Diferentes De La Sierra,36,artist,spotify:artist:3fD1A7V9jV0eImdBL0Bcux
14697,293,https://open.spotify.com/artist/34nbQa7Hug9DYk...,None,2468682,"[corrido, corridos tumbados, musica mexicana, ...",https://api.spotify.com/v1/artists/34nbQa7Hug9...,34nbQa7Hug9DYkRJpfKNFv,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",T3R Elemento,73,artist,spotify:artist:34nbQa7Hug9DYkRJpfKNFv
14698,293,https://open.spotify.com/artist/63zgw9LU9aqnuH...,None,132013,"[corrido, corridos alternativos, corridos clas...",https://api.spotify.com/v1/artists/63zgw9LU9aq...,63zgw9LU9aqnuHnvHr3ZUZ,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Arley Perez,50,artist,spotify:artist:63zgw9LU9aqnuHnvHr3ZUZ


# Serialización

In [11]:
df_final.to_csv("../data/{}.csv.gz".format(file_name), 
           index=False, 
           compression="gzip")